<div align="center"><span style="font-family:arialblack;font-size:48px">Traitement du fichier des achats clients pour analyse SQL et Power BI</span></div><hr>


<div><span style="font-family:arialblack;font-size:24px">Le but de ce notebook est de vérifier les informations d'achats clients contenues dans un fichier csv puis de les nettoyer et transformer pour être facilement utilisables dans un rapport Power BI</span></div>
<div><span style="font-family:arialblack;font-size:24px">Nous stockerons ces données dans une table MySQL qui servira de source dans Power Query</span></div>

<hr>

In [2]:
# Chargement des données sources (csv) à l'aide de pandas

import pandas as pd

df = pd.read_csv('C:\Data Analyst\Etude de cas\Shopping\customer_shopping_behavior.csv')

In [3]:
# Visualisation des 5 premiers enregistrements du dataset

df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


In [4]:
# Visualisation des infos de chaque colonne (nom, nb de valeurs non-nulles, type de donnée)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Customer ID             3900 non-null   int64  
 1   Age                     3900 non-null   int64  
 2   Gender                  3900 non-null   object 
 3   Item Purchased          3900 non-null   object 
 4   Category                3900 non-null   object 
 5   Purchase Amount (USD)   3900 non-null   int64  
 6   Location                3900 non-null   object 
 7   Size                    3900 non-null   object 
 8   Color                   3900 non-null   object 
 9   Season                  3900 non-null   object 
 10  Review Rating           3863 non-null   float64
 11  Subscription Status     3900 non-null   object 
 12  Shipping Type           3900 non-null   object 
 13  Discount Applied        3900 non-null   object 
 14  Promo Code Used         3900 non-null   

In [5]:
# Statistiques globales

df.describe(include='all')

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
count,3900.000000,3900.000000,3900,3900,3900,3900.000000,3900,3900,3900,3900,3863.000000,3900,3900,3900,3900,3900.000000,3900,3900
unique,NaN,NaN,2,25,4,NaN,50,4,25,4,NaN,2,6,2,2,NaN,6,7
top,NaN,NaN,Male,Blouse,Clothing,NaN,Montana,M,Olive,Spring,NaN,No,Free Shipping,No,No,NaN,PayPal,Every 3 Months
freq,NaN,NaN,2652,171,1737,NaN,96,1755,177,999,NaN,2847,675,2223,2223,NaN,677,584
mean,1950.500000,44.068462,NaN,NaN,NaN,59.764359,NaN,NaN,NaN,NaN,3.750065,NaN,NaN,NaN,NaN,25.351538,NaN,NaN
std,1125.977353,15.207589,NaN,NaN,NaN,23.685392,NaN,NaN,NaN,NaN,0.716983,NaN,NaN,NaN,NaN,14.447125,NaN,NaN
min,1.000000,18.000000,NaN,NaN,NaN,20.000000,NaN,NaN,NaN,NaN,2.500000,NaN,NaN,NaN,NaN,1.000000,NaN,NaN
25%,975.750000,31.000000,NaN,NaN,NaN,39.000000,NaN,NaN,NaN,NaN,3.100000,NaN,NaN,NaN,NaN,13.000000,NaN,NaN
50%,1950.500000,44.000000,NaN,NaN,NaN,60.000000,NaN,NaN,NaN,NaN,3.800000,NaN,NaN,NaN,NaN,25.000000,NaN,NaN
75%,2925.250000,57.000000,NaN,NaN,NaN,81.000000,NaN,NaN,NaN,NaN,4.400000,NaN,NaN,NaN,NaN,38.000000,NaN,NaN


In [6]:
# Vérification de la présence de données nulles ou absentes dans le dataset

df.isnull().sum()

Customer ID                0
Age                        0
Gender                     0
Item Purchased             0
Category                   0
Purchase Amount (USD)      0
Location                   0
Size                       0
Color                      0
Season                     0
Review Rating             37
Subscription Status        0
Shipping Type              0
Discount Applied           0
Promo Code Used            0
Previous Purchases         0
Payment Method             0
Frequency of Purchases     0
dtype: int64

In [7]:
# Remplacement des valeurs manquantes dans 'Review Rating' par la valeur médiane par 'Category'

df['Review Rating'] = df.groupby('Category')['Review Rating'].transform(lambda x: x.fillna(x.median()))

In [7]:
df.isnull().sum()

Customer ID               0
Age                       0
Gender                    0
Item Purchased            0
Category                  0
Purchase Amount (USD)     0
Location                  0
Size                      0
Color                     0
Season                    0
Review Rating             0
Subscription Status       0
Shipping Type             0
Discount Applied          0
Promo Code Used           0
Previous Purchases        0
Payment Method            0
Frequency of Purchases    0
dtype: int64

In [8]:
# Homogénéisation des noms de colonnes : minuscules, sans espace

df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ','_')
df = df.rename(columns={'purchase_amount_(usd)':'purchase_amount'})

In [9]:
# Liste des noms de colonnes

df.columns

Index(['customer_id', 'age', 'gender', 'item_purchased', 'category',
       'purchase_amount', 'location', 'size', 'color', 'season',
       'review_rating', 'subscription_status', 'shipping_type',
       'discount_applied', 'promo_code_used', 'previous_purchases',
       'payment_method', 'frequency_of_purchases'],
      dtype='object')

In [10]:
# Création d'une nouvelle colonne 'age_group' pour segmentation de l'âge en 4 quartiles

labels = ['Young Adult', 'Adult', 'Middle-aged', 'Senior']
df['age_group'] = pd.qcut(df['age'], q=4, labels = labels)

In [11]:
# Affichage des 10 premiers enregistrements pour les colonnes age et age_group

df[['age','age_group']].head(10)

,age,age_group
0,55,Middle-aged
1,19,Young Adult
2,50,Middle-aged
3,21,Young Adult
4,45,Middle-aged
5,46,Middle-aged
6,63,Senior
7,27,Young Adult
8,26,Young Adult
9,57,Middle-aged


In [12]:
# Création d'une nouvelle colonne purchase_frequency_days qui traduit la valeur textuelle
# de la colonne 'frequency_of_purchases' en valeur numérique grâce à un mapping

frequency_mapping = {
    'Fortnightly': 14,
    'Weekly': 7,
    'Monthly': 30,
    'Quarterly': 90,
    'Bi-Weekly': 14,
    'Annually': 365,
    'Every 3 Months': 90
}

df['purchase_frequency_days'] = df['frequency_of_purchases'].map(frequency_mapping)

In [13]:
# Affichage des 10 premiers enregistrements pour les colonnes 'purchase_frequency_days' et 'frequency_of_purchases'

df[['purchase_frequency_days','frequency_of_purchases']].head(10)

,purchase_frequency_days,frequency_of_purchases
0,14,Fortnightly
1,14,Fortnightly
2,7,Weekly
3,7,Weekly
4,365,Annually
5,7,Weekly
6,90,Quarterly
7,7,Weekly
8,365,Annually
9,90,Quarterly


In [14]:
# Affichage des 10 premiers enregistrements pour les colonnes 'discount_applied' et 'promo_code_used'

df[['discount_applied','promo_code_used']].head(10)

,discount_applied,promo_code_used
0,Yes,Yes
1,Yes,Yes
2,Yes,Yes
3,Yes,Yes
4,Yes,Yes
5,Yes,Yes
6,Yes,Yes
7,Yes,Yes
8,Yes,Yes
9,Yes,Yes


In [15]:
# D'après la visualisation précédente, il semblerait que 'discount_applied' et 'promo_code_used'
# soient identiques, voyons si nos observations sont exactes :

(df['discount_applied'] == df['promo_code_used']).all()

np.True_

In [16]:
# Notre perception première s'avère exacte, 'discount_applied' et 'promo_code_used' disposent des même valeurs
# pour tous les enregistrements, on peut donc supprimer l'une des 2 colonnes ('promo_code_used')

df = df.drop('promo_code_used', axis=1)

In [17]:
# Liste des noms de colonnes (la colonne 'promo_code_used' a bien été supprimée)

df.columns

Index(['customer_id', 'age', 'gender', 'item_purchased', 'category',
       'purchase_amount', 'location', 'size', 'color', 'season',
       'review_rating', 'subscription_status', 'shipping_type',
       'discount_applied', 'previous_purchases', 'payment_method',
       'frequency_of_purchases', 'age_group', 'purchase_frequency_days'],
      dtype='object')

## Code pour MySQL

In [24]:
# Installation des librairies pour MySQL

!pip install pymysql sqlalchemy


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
# Création de la bdd 'customer_behavior' ainsi que sa table 'customer'
# qui contiendra les données de notre dataset

from sqlalchemy import create_engine

# Connection MySQL : indiquer votre username/password
username = ""
password = ""
host = "localhost"
port = "3306"
database = "customer_behavior"

# Création de la bdd
engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")

# Export des données du dataframe vers la table 'customer'
table_name = "customer"   # choose any table name
df.to_sql(table_name, engine, if_exists="replace", index=False)

# Extraction des 5 premiers enregistrements de la table 'customer' pour vérification
pd.read_sql("SELECT * FROM customer LIMIT 5;", engine)

,customer_id,age,gender,item_purchased,category,purchase_amount,location,size,color,season,review_rating,subscription_status,shipping_type,discount_applied,previous_purchases,payment_method,frequency_of_purchases,age_group,purchase_frequency_days
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,14,Venmo,Fortnightly,Middle-aged,14
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,2,Cash,Fortnightly,Young Adult,14
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,23,Credit Card,Weekly,Middle-aged,7
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,49,PayPal,Weekly,Young Adult,7
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,31,PayPal,Annually,Middle-aged,365


In [27]:
# Quel est le revenu total généré par les femmes & par les hommes ?
sql_inst = 'select gender, SUM(purchase_amount) as revenue from customer group by gender;'

pd.read_sql(sql_inst, engine)


,gender,revenue
0,Male,157890.0
1,Female,75191.0


In [29]:
# Quels sont les clients qui ont utilisé une remise mais qui ont dépensé plus que la dépense moyenne globale ?
sql_inst = "select customer_id, purchase_amount from customer where discount_applied = 'Yes' and purchase_amount >= (select AVG(purchase_amount) from customer) order by purchase_amount desc;"
pd.read_sql(sql_inst, engine)

,customer_id,purchase_amount
0,43,100
1,96,100
2,194,100
3,205,100
4,244,100
...,...,...
834,1247,60
835,1296,60
836,1333,60
837,1424,60


In [31]:
# Quels sont les 5 produits avec la meilleure moyenne d'évaluation ?

sql_inst = "select item_purchased, round(avg(review_rating),2) as Average_Product_Rating from customer group by item_purchased order by avg(review_rating) desc limit 5;"
pd.read_sql(sql_inst, engine)

,item_purchased,Average_Product_Rating
0,Gloves,3.86
1,Sandals,3.84
2,Boots,3.82
3,Hat,3.80
4,Skirt,3.78


In [33]:
# Comparer la moyenne des achats en livraison 'standard' et en livraison 'express'
sql_inst = "select shipping_type, ROUND(AVG(purchase_amount),2) as avg_purchase_amount from customer where shipping_type in ('Standard','Express') group by shipping_type;"
pd.read_sql(sql_inst, engine)

,shipping_type,avg_purchase_amount
0,Express,60.48
1,Standard,58.46


In [34]:
# Les clients inscrits dépensent-ils plus ?

sql_inst = "SELECT subscription_status, COUNT(customer_id) AS total_customers, ROUND(AVG(purchase_amount),2) AS avg_spend, ROUND(SUM(purchase_amount),2) AS total_revenue FROM customer GROUP BY subscription_status ORDER BY total_revenue,avg_spend DESC;"
pd.read_sql(sql_inst, engine)


,subscription_status,total_customers,avg_spend,total_revenue
0,Yes,1053,59.49,62645.0
1,No,2847,59.87,170436.0


In [38]:
# Quels sont les 5 produits avec le plus grand nb d'achats remisés ?

sql_inst = "SELECT item_purchased, ROUND(100.0 * SUM(CASE WHEN discount_applied = 'Yes' THEN 1 ELSE 0 END)/COUNT(*),2) AS discount_rate FROM customer GROUP BY item_purchased ORDER BY discount_rate DESC LIMIT 5;"
pd.read_sql(sql_inst, engine)


,item_purchased,discount_rate
0,Hat,50.00
1,Sneakers,49.66
2,Coat,49.07
3,Sweater,48.17
4,Pants,47.37


In [39]:
# Segmenter les types de clients par nb achats et les compter

sql_inst = "with customer_type as (SELECT customer_id, previous_purchases,CASE WHEN previous_purchases = 1 THEN 'New' WHEN previous_purchases BETWEEN 2 AND 10 THEN 'Returning' ELSE 'Loyal' END AS customer_segment FROM customer) select customer_segment,count(*) AS Number_of_Customers from customer_type group by customer_segment;"
pd.read_sql(sql_inst, engine)

,customer_segment,Number_of_Customers
0,Loyal,3116
1,Returning,701
2,New,83


In [40]:
# Quels sont les 3 produits les plus achetés de chaque catégorie ?

sql_inst = "WITH item_counts AS (SELECT category, item_purchased, COUNT(customer_id) AS total_orders, ROW_NUMBER() OVER (PARTITION BY category ORDER BY COUNT(customer_id) DESC) AS item_rank FROM customer GROUP BY category, item_purchased) SELECT item_rank,category, item_purchased, total_orders FROM item_counts WHERE item_rank <=3;"
pd.read_sql(sql_inst, engine)

,item_rank,category,item_purchased,total_orders
0,1,Accessories,Jewelry,171
1,2,Accessories,Sunglasses,161
2,3,Accessories,Belt,161
3,1,Clothing,Blouse,171
4,2,Clothing,Pants,171
5,3,Clothing,Shirt,169
6,1,Footwear,Sandals,160
7,2,Footwear,Shoes,150
8,3,Footwear,Sneakers,145
9,1,Outerwear,Jacket,163


In [41]:
# Par statut de souscription, nb de clients qui ont effectué plus de 5 achats :

sql_inst = "SELECT subscription_status, COUNT(customer_id) AS repeat_buyers FROM customer WHERE previous_purchases > 5 GROUP BY subscription_status;"
pd.read_sql(sql_inst, engine)

,subscription_status,repeat_buyers
0,Yes,958
1,No,2518


In [43]:
# Quel montant d'achat par groupe d'âge ?

sql_inst = "SELECT age_group, SUM(purchase_amount) AS total_revenue FROM customer GROUP BY age_group ORDER BY total_revenue desc;"
pd.read_sql(sql_inst, engine)


,age_group,total_revenue
0,Young Adult,62143.0
1,Middle-aged,59197.0
2,Adult,55978.0
3,Senior,55763.0


In [47]:
# Min et Max âge des groupes ?

sql_inst = "SELECT age_group as GROUPE, min(age) as AGE_MIN, max(age) as AGE_MAX FROM customer GROUP BY GROUPE ORDER BY AGE_MIN;"
pd.read_sql(sql_inst, engine)


,GROUPE,AGE_MIN,AGE_MAX
0,Young Adult,18,31
1,Adult,32,44
2,Middle-aged,45,57
3,Senior,58,70


In [52]:
# Commande la plus forte des groupes ?

sql_inst = "SELECT MIN(purchase_amount) AS MNT_MIN, MAX(purchase_amount) AS MNT_MAX, COUNT(purchase_amount) AS NB_CMDES, age_group as GROUPE FROM customer GROUP BY GROUPE ORDER BY MNT_MAX DESC;"
df1 = pd.read_sql(sql_inst, engine)

In [54]:
# NB total de commandes dans total

sql_inst = "SELECT COUNT(purchase_amount) AS NB_TOTAL FROM customer"
total = pd.read_sql(sql_inst, engine)


In [67]:
# Passage de ce NB total dans une variable de type numérique
tot = int(total['NB_TOTAL'])
tot

C:\Users\joseg\AppData\Local\Temp\ipykernel_24144\1606475779.py:1: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  tot = int(total['NB_TOTAL'])


3900

In [82]:
# Nouvelle colonne POURCENTAGE dans la dataframe
df1['POURCENTAGE'] = round((df1['NB_CMDES']/tot) * 100, 2)
df1[['GROUPE', 'POURCENTAGE']]


,GROUPE,POURCENTAGE
0,Middle-aged,25.28
1,Young Adult,26.36
2,Senior,24.21
3,Adult,24.15
